In [1]:
from tomato.symbolic.symbtranalyzer import SymbTrAnalyzer
import os
from pprint import pprint


In [2]:
# symbtr name, 
# it is needed if the filename is modified from the SymbTr naming convention
symbtr_name = 'ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede'

# score filepaths
txt_filename = os.path.join(symbtr_name, symbtr_name + '.txt')
mu2_filename = os.path.join(symbtr_name, symbtr_name + '.mu2')

# instantiate analyzer object
scoreAnalyzer = SymbTrAnalyzer(verbose=True)


You can use the single line call "analyze," which does all the available analysis simultaneously

In [3]:
score_data = scoreAnalyzer.analyze(txt_filename, None, symbtr_name=symbtr_name)


- Automatic phrase segmentation on the SymbTr-txt file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt
  The call took 9.23 seconds to execute.


In [4]:
# pretty print the metadata
pprint(score_data['metadata'])


{'composer': {'mbid': 'a6775723-69b6-4997-82da-bf230b1f8655',
              'name': 'Neyzen Aziz Dede',
              'symbtr_slug': 'neyzen_aziz_dede'},
 'duration': {'unit': 'second', 'value': 275.821},
 'form': {'attribute_key': 'sazsemaisi',
          'mb_attribute': 'Sazsemaisi',
          'source': 'http://musicbrainz.org/work/e7924b0d-c8a0-4b4a-b253-8eec898eac1e',
          'symbtr_slug': 'sazsemaisi'},
 'language': 'zxx',
 'lyricist': {},
 'makam': {'attribute_key': 'ussak',
           'mb_attribute': 'Uşşak',
           'source': 'http://musicbrainz.org/work/e7924b0d-c8a0-4b4a-b253-8eec898eac1e',
           'symbtr_slug': 'ussak'},
 'number_of_notes': 680,
 'recordings': [{'mbid': '9a7d391f-90a6-4f2d-a0d2-5e6a5365b149',
                 'title': 'Uşşak Saz Semaisi'},
                {'mbid': '9b869a16-0ab0-43a2-a51e-2a03e9484e70',
                 'title': 'Uşşak Saz Semaisi'},
                {'mbid': '86721a3a-268f-4422-8803-73a196d2a571',
                 'title': 'Uşşak Sa

... or you can call all the methods individually


In [5]:
from tomato.metadata.symbtr import SymbTr as SymbTrMetadata
from tomato.symbolic.symbtr.reader.txt import TxtReader
from tomato.symbolic.symbtr.reader.mu2 import Mu2Reader
from tomato.symbolic.symbtr.dataextractor import DataExtractor
from tomato.symbolic.symbtr.section import SectionExtractor
from tomato.symbolic.symbtr.segment import SegmentExtractor
from tomato.symbolic.symbtr.rhythmicfeature import RhythmicFeatureExtractor

# relevant recording or work mbid, if you want additional information from musicbrainz
# Note 1: MBID input will make the function returns significantly slower because we
#         have to wait a couple of seconds before each subsequent query from MusicBrainz.
# Note 2: very rare but there can be more that one mbid returned. We are going to use 
#         the first work to get fetch the metadata
mbid = SymbTrMetadata.get_mbids_from_symbtr_name(symbtr_name)[0]

# read the txt score
txt_score, is_score_content_valid = TxtReader.read(
    txt_filename, symbtr_name=symbtr_name)

# read metadata from musicbrainz
mb_metadata, is_mb_metadata_valid = SymbTrMetadata.from_musicbrainz(
    symbtr_name, mbid=score_data['mbid'])

# add duration & number of notes
mb_metadata['duration'] = {
    'value': sum(score_data['score']['duration']) * 0.001, 'unit': 'second'}
mb_metadata['number_of_notes'] = len(txt_score['duration'])

# read metadata from the mu2 header
mu2_header, header_row, is_mu2_header_valid = Mu2Reader.read_header(
    mu2_filename, symbtr_name=symbtr_name)

# merge metadata
score_metadata = DataExtractor.merge(mb_metadata, mu2_header)

# sections
section_extractor = SectionExtractor()
sections, is_section_data_valid = section_extractor.from_txt_score(
    txt_score, symbtr_name)

# annotated phrases
segment_extractor = SegmentExtractor()
phrase_annotations = segment_extractor.extract_phrases(
    txt_score, sections=sections)

# Automatic phrase segmentation on the SymbTr-txt score using pre-trained model
segment_bounds = scoreAnalyzer.segment_phrase(txt_filename, symbtr_name=symbtr_name)
segments = segment_extractor.extract_segments(
    txt_score,
    segment_bounds['boundary_note_idx'],
    sections=sections)

# rhythmic structure
rhythmic_structure = RhythmicFeatureExtractor.extract_rhythmic_structure(
    txt_score)

- Automatic phrase segmentation on the SymbTr-txt file: ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede/ussak--sazsemaisi--aksaksemai----neyzen_aziz_dede.txt
  The call took 8.07 seconds to execute.
